In [1]:
from utilities.task_service import get_authentication_token, get_task, send_answer
from auth import apikey
from openai import OpenAI # to zadanie z przyzwyczajenia zrobiłam bezpośrednio w OpenAI a nie w LangChain

from dotenv import load_dotenv
load_dotenv(override=True)

task_name = "blogger"
client = OpenAI()

In [2]:
# autoryzacja
token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token")
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token


In [3]:
# pobranie danych do zadania
response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'please write blog post for the provided outline', 'blog': ['Wstęp: kilka słów na temat historii pizzy', 'Niezbędne składniki na pizzę', 'Robienie pizzy', 'Pieczenie pizzy w piekarniku']}


In [4]:
print(response['blog'])

['Wstęp: kilka słów na temat historii pizzy', 'Niezbędne składniki na pizzę', 'Robienie pizzy', 'Pieczenie pizzy w piekarniku']


In [5]:
def text_generator(context, prompt):
    completion = client.chat.completions.create(
            model="gpt-4",
            temperature = 0.7,
            messages=[
                    {"role": "system", "content": context},
                    {"role": "user", "content": prompt}
                ]
            )
    return completion.choices[0].message.content

In [6]:
article = []
section_done = []

context = """You are an assistant to create articles for the website. You don't create the whole article right away, but the user sends you the headline of the section to write, and you create it.In addition, the user sends you a list of headings from the article that have already been written. Don't duplicate the information they focused on. Some tips you should follow:
    - focus on creating content that is valuable and well thought out;
    - avoid plagiarism and be creative. Create unique content that will make your text stand out;
    - learn the basics of search engine optimization (SEO) and skillfully use keywords to increase your content's visibility in search results;
    - when writing, be clear and concise. Avoid unnecessary words and complex sentence constructions that can put off readers;
    - use bullets and highlighting to make your content easier to read and navigate. Good structure helps readers quickly find what they are looking for;
    - encourage interaction through questions or calls to action; 
    - use a friendly, conversational tone;"""

for section in response['blog']:
    prompt = f"Stwórz treść sekcji: {section}. Rozwinąłeś już wcześniej takie sekcje: {section_done}."

    section_text = text_generator(context, prompt)
    section_done.append(section)
    article.append(section_text)

In [7]:
# wysłanie odpowiedzi
response = send_answer(token, article)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


In [9]:
print(*article)

Wstęp: Kilka Słów na Temat Historii Pizzy

Chyba nie ma osoby, która nie słyszałaby o pizzy, tym kulinarnym fenomenie, który zdobył serca (i żołądki) ludzi na całym świecie. Ale czy kiedykolwiek zastanawialiście się, skąd wzięło się to wyjątkowe danie? Cóż, zanurzmy się w fascynującą historię pizzy, pełną smaku, tradycji i... nieco kontrowersji.

Pierwsze wzmianki na temat pizzy sięgają starożytnego Egiptu i Grecji, gdzie piekło się chleby płaskie, podobne do dzisiejszych pizz, choć oczywiście bez charakterystycznych dodatków, które znamy dzisiaj. Prawdziwą pizzę, jaką znamy, narodził się jednak w Neapolu, we Włoszech, gdzie w XVIII wieku prosti ludzie zaczęli dodawać do swoich chlebów płaskich pomidory - owoc, który wówczas był w Europie nowością.

Prawdziwy przełom w historii pizzy nastąpił w XIX wieku, kiedy to Raffaele Esposito, znany neapolitański piekarz, stworzył pizzę Margheritę na cześć królowej Margherity. Ta prosta pizza, składająca się z pomidorów, mozzarelli i bazylii, sta